In [ ]:
import pandas as pd
import numpy as np
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score
)
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import os
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Carregando dados de treino e teste
train_df = pd.read_csv('../references/exemplo_train.csv')
test_df = pd.read_csv('../references/exemplo_test.csv')

print(f"Dados de treino: {train_df.shape}")
print(f"Dados de teste: {test_df.shape}")

# Visualizando primeiras linhas
train_df.head()


In [ ]:
# Verificando informações gerais
print("Informações dos dados de treino:")
print(train_df.info())
print("\nValores únicos da variável alvo:")
print(train_df['Credit_Score'].value_counts())
print("\nDistribuição percentual:")
print(train_df['Credit_Score'].value_counts(normalize=True) * 100)


In [ ]:
# Verificando valores nulos
print("Valores nulos por coluna:")
print(train_df.isnull().sum())
print("\nPercentual de valores nulos:")
print((train_df.isnull().sum() / len(train_df)) * 100)


In [ ]:
def preprocess_data(df, is_training=True):
    """
    Função para pré-processar os dados de credit score
    """
    # Cópia do dataframe
    df_processed = df.copy()
    
    # Removendo colunas desnecessárias
    columns_to_drop = ['ID', 'Customer_ID', 'Name', 'SSN']
    df_processed = df_processed.drop(columns=[col for col in columns_to_drop if col in df_processed.columns])
    
    # Tratamento de valores problemáticos em colunas numéricas
    numeric_columns = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
                      'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date',
                      'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries',
                      'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month',
                      'Amount_invested_monthly', 'Monthly_Balance']
    
    for col in numeric_columns:
        if col in df_processed.columns:
            # Convertendo valores não numéricos para NaN
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            
            # Tratando outliers negativos para idade
            if col == 'Age':
                df_processed[col] = df_processed[col].apply(lambda x: np.nan if x < 0 or x > 100 else x)
    
    # Tratamento de valores categóricos problemáticos
    categorical_columns = ['Month', 'Occupation', 'Type_of_Loan', 'Credit_Mix', 
                          'Credit_History_Age', 'Payment_of_Min_Amount', 'Payment_Behaviour']
    
    for col in categorical_columns:
        if col in df_processed.columns:
            # Substituindo valores problemáticos por NaN
            df_processed[col] = df_processed[col].replace(['_', '!@9#%8', '#F%$D@*&8', '_______', 'NA'], np.nan)
    
    return df_processed

# Aplicando pré-processamento
train_processed = preprocess_data(train_df, is_training=True)
test_processed = preprocess_data(test_df, is_training=False)

print("Dados após pré-processamento:")
print(f"Treino: {train_processed.shape}")
print(f"Teste: {test_processed.shape}")
print("\nValores nulos após limpeza:")
print(train_processed.isnull().sum())


In [ ]:
# Separando features e target
features = list(train_processed.columns)
features.remove('Credit_Score')

X = train_processed[features]
y = train_processed['Credit_Score']

print(f"Número de features: {len(features)}")
print(f"Features: {features}")
print(f"\nDistribuição da variável alvo:")
print(y.value_counts())


In [ ]:
# Criando pipeline de pré-processamento
# Identificando colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=[object]).columns.tolist()

print(f"Features numéricas: {numeric_features}")
print(f"Features categóricas: {categorical_features}")

# Pipeline de pré-processamento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Dividindo dados para treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"\nTamanhos após divisão:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")


In [ ]:
# Configuração do MLflow (sem autolog para controle manual)
dagshub.init(repo_owner="domires", repo_name="fiap-mlops-score-model", mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/domires/fiap-mlops-score-model.mlflow")

print("MLflow configurado (sem autolog automático)!")


In [ ]:
def evaluate_and_log_classification_model(model_name, pipeline, X_val, y_val):
    """
    Função para avaliar e registrar modelo Random Forest de classificação
    """
    # Fazendo predições
    predictions = pipeline.predict(X_val)
    prediction_proba = pipeline.predict_proba(X_val) if hasattr(pipeline, "predict_proba") else None
    
    # Calculando métricas
    accuracy = accuracy_score(y_val, predictions)
    precision = precision_score(y_val, predictions, average='weighted')
    recall = recall_score(y_val, predictions, average='weighted')
    f1 = f1_score(y_val, predictions, average='weighted')
    
    # AUC-ROC para classificação multiclasse
    if prediction_proba is not None:
        try:
            auc_roc = roc_auc_score(y_val, prediction_proba, multi_class='ovr', average='weighted')
        except:
            auc_roc = 0.0
    else:
        auc_roc = 0.0
    
    # Registrando métricas no MLflow
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("auc_roc", auc_roc)
    
    # Criando assinatura do modelo
    signature = infer_signature(X_val, predictions)
    
    # Registrando modelo sklearn
    mlflow.sklearn.log_model(pipeline, model_name, signature=signature, input_example=X_val[:5])
    
    print(f"Modelo {model_name} avaliado:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC-ROC: {auc_roc:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc
    }


In [ ]:
with mlflow.start_run(run_name="Random Forest - Credit Score Classification"):
    # Criando pipeline com Random Forest usando parâmetros otimizados da referência
    final_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=150,
            criterion='gini', 
            max_features='sqrt',
            random_state=42,
            max_depth=30,
            class_weight='balanced_subsample',
            min_samples_split=2,
            min_samples_leaf=1
        ))
    ])
    
    # Registrando parâmetros do modelo fixo
    mlflow.log_param("n_estimators", 150)
    mlflow.log_param("criterion", "gini")
    mlflow.log_param("max_features", "sqrt")
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("class_weight", "balanced_subsample")
    mlflow.log_param("min_samples_split", 2)
    mlflow.log_param("random_state", 42)
    
    print("Treinando modelo Random Forest com parâmetros otimizados...")
    final_model.fit(X_train, y_train)
    
    # Avaliando modelo
    print("\nAvaliando modelo Random Forest...")
    metrics = evaluate_and_log_classification_model("random_forest_credit_score", final_model, X_val, y_val)
    
    print(f"\nModelo Random Forest treinado com sucesso!")
    print(f"Acurácia de validação: {metrics['accuracy']:.4f}")


In [ ]:
# Preparando dados de teste para predição
if 'Credit_Score' in test_processed.columns:
    # Se o conjunto de teste tiver a variável alvo (para avaliação)
    X_test = test_processed[features]
    y_test = test_processed['Credit_Score']
    
    # Fazendo predições no conjunto de teste
    test_predictions = final_model.predict(X_test)
    test_probabilities = final_model.predict_proba(X_test)
    
    # Avaliando no conjunto de teste
    test_accuracy = accuracy_score(y_test, test_predictions)
    test_precision = precision_score(y_test, test_predictions, average='weighted')
    test_recall = recall_score(y_test, test_predictions, average='weighted')
    test_f1 = f1_score(y_test, test_predictions, average='weighted')
    
    print("=== RESULTADOS NO CONJUNTO DE TESTE ===")
    print(f"Acurácia: {test_accuracy:.4f}")
    print(f"Precisão: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"F1-Score: {test_f1:.4f}")
    
    # Matriz de confusão
    from sklearn.metrics import classification_report
    print("\nRelatório de classificação:")
    print(classification_report(y_test, test_predictions))
    
else:
    # Se não houver variável alvo no teste, apenas fazer predições
    X_test = test_processed[features]
    test_predictions = final_model.predict(X_test)
    test_probabilities = final_model.predict_proba(X_test)
    
    print("Predições realizadas no conjunto de teste.")
    print(f"Distribuição das predições:")
    unique, counts = np.unique(test_predictions, return_counts=True)
    for label, count in zip(unique, counts):
        print(f"  {label}: {count} ({count/len(test_predictions)*100:.1f}%)")

# Salvando predições
predictions_df = pd.DataFrame({
    'prediction': test_predictions
})

# Adicionando probabilidades se disponível
if test_probabilities is not None:
    prob_columns = [f'prob_class_{i}' for i in range(test_probabilities.shape[1])]
    prob_df = pd.DataFrame(test_probabilities, columns=prob_columns)
    predictions_df = pd.concat([predictions_df, prob_df], axis=1)

print(f"\nPrimeiras 10 predições:")
print(predictions_df.head(10))


In [ ]:
# Salvando o modelo treinado localmente
import joblib
import os

# Criando diretório para modelos se não existir
models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)

# Salvando o modelo final
model_path = os.path.join(models_dir, 'credit_score_random_forest.pkl')
joblib.dump(final_model, model_path)

print(f"Modelo Random Forest salvo em: {model_path}")

# Salvando também as predições
predictions_path = os.path.join(models_dir, 'test_predictions.csv')
predictions_df.to_csv(predictions_path, index=False)

print(f"Predições salvas em: {predictions_path}")

# Salvando informações do modelo
model_info = {
    'model_type': 'RandomForestClassifier',
    'features': features,
    'validation_metrics': metrics,
    'model_params': final_model.named_steps['classifier'].get_params()
}

import json
info_path = os.path.join(models_dir, 'model_info.json')
with open(info_path, 'w') as f:
    json.dump(model_info, f, indent=2, default=str)

print(f"Informações do modelo salvas em: {info_path}")
print("\n=== RESUMO DO MODELO FINAL ===")
print(f"Tipo: Random Forest Classifier")
print(f"Número de features: {len(features)}")
print(f"Métricas de validação:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")


In [ ]:
# Verificando experimentos registrados no MLflow
print("=== VERIFICAÇÃO DE EXPERIMENTOS NO MLFLOW ===")
try:
    # Listando os últimos experimentos
    experiments = mlflow.search_experiments()
    if experiments:
        latest_exp = experiments[0]
        print(f"Experimento atual: {latest_exp.name}")
        
        # Listando as runs mais recentes
        runs = mlflow.search_runs(experiment_ids=[latest_exp.experiment_id], max_results=5)
        print(f"\nÚltimas {len(runs)} execuções:")
        for i, run in runs.iterrows():
            print(f"  - {run['tags.mlflow.runName']} (Status: {run['status']})")
    
    print(f"\nApenas 1 modelo Random Forest foi treinado nesta execução!")
    
except Exception as e:
    print(f"Não foi possível listar experimentos: {e}")
    print("Mas apenas 1 modelo Random Forest foi treinado!")


In [ ]:
# Visualizando importância das features
import matplotlib.pyplot as plt
import seaborn as sns

# Obtendo importância das features do Random Forest
rf_model = final_model.named_steps['classifier']

# Obtendo nomes das features após o pré-processamento
feature_names = (numeric_features + 
                list(final_model.named_steps['preprocessor']
                    .named_transformers_['cat']
                    .named_steps['onehot']
                    .get_feature_names_out(categorical_features)))

# Criando DataFrame com importâncias
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plotando as 15 features mais importantes
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
plt.title('Top 15 Features mais Importantes - Random Forest')
plt.xlabel('Importância')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

# Mostrando as 10 features mais importantes numericamente
print("=== TOP 10 FEATURES MAIS IMPORTANTES ===")
for i, (idx, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<30} {row['importance']:.4f}")

# Visualizando matriz de confusão se temos dados de teste com target
if 'Credit_Score' in test_processed.columns:
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    
    cm = confusion_matrix(y_test, test_predictions)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=np.unique(y_test), 
                yticklabels=np.unique(y_test))
    plt.title('Matriz de Confusão - Conjunto de Teste')
    plt.xlabel('Predições')
    plt.ylabel('Valores Reais')
    plt.tight_layout()
    plt.show()
